## Przeszukiwanie tekstu - algorytmy wyszukiwarek internetowych

#### prezentuje Mateusz Puto

## 1. Przeszukiwanie plików

In [1]:
def bold(string):
    return "\033[1m" + string + "\033[0;0m"

In [2]:
"""https://czytac.com/java-book/book/20_000_mil_podmorskiej_zeglugi"""

# Wyszukiwanie samozakańczające
dwa_tysiace_mil_podmorskiej_zeglugi = open('20_000_mil_podmorskiej_zeglugi.txt', 'r')

for sentence in dwa_tysiace_mil_podmorskiej_zeglugi:
    if "Nautilus" in sentence:
        print(sentence)
        break

# Wyszukiwanie pełne
dwa_tysiace_mil_podmorskiej_zeglugi = open('20_000_mil_podmorskiej_zeglugi.txt', 'r')

nautilus_mentions = []
for sentence in dwa_tysiace_mil_podmorskiej_zeglugi:
    if "Nautilus" in sentence:
        nautilus_mentions.append(sentence)

print(bold(str(len(nautilus_mentions))), end="\n\n")

# Efekt niedawności (ang. "recency effect")
print(bold("Po przeczytaniu książki pamiętamy to: \n") + nautilus_mentions[0] + bold(" i to [spoiler !]:\n") + nautilus_mentions[-1])

— Dla pana — odpowiedział dowódca — jestem kapitanem Nemo; pan zaś i pańscy towarzysze jesteście dla mnie podróżnymi na „Nautilusie”.

564

Po przeczytaniu książki pamiętamy to: 
— Dla pana — odpowiedział dowódca — jestem kapitanem Nemo; pan zaś i pańscy towarzysze jesteście dla mnie podróżnymi na „Nautilusie”.
 i to [spoiler !]:
Spodziewam się, że tak będzie. Mam nadzieję, że potężny „Nautilus” zwyciężył morze w najstraszniejszej jego otchłani, że przetrwał to, czego żaden okręt przetrwać nie zdołał. Jeśli tak jest, jeśli kapitan Nemo żegluje jeszcze po morzach, które mu są przybraną ojczyzną, oby uczucie zemsty wygasło w dzikim jego sercu! Oby ją wygoniło z jego myśli rozglądanie się w tylu cudach! Wykonawca sprawiedliwości niech ustąpi miejsca uczonemu dokonywającemu spokojnych poszukiwań podmorskich. Jeśli los jego jest niezwykły, to i wspaniały zarazem. Sam to osobiście pojąłem podczas dziesięciomiesięcznego, nadnaturalnego prawie mego istnienia.



[Miary jakości - dokładność, precyzja i czułość](https://www.mimuw.edu.pl/~wjaworski/SU/SU04_miary_jakosci.pdf)


Macierz błędów (ang. "Confusion matrix"):

                                                                  Przewidywanie

                                                         Dopasowanie         Brak dopasowania

                                ---------------------------------------------------------------------

                                Dopasowanie             True Positive        False Positive
        Stan rzeczywisty                         
                                          
                                Brak dopasowania        False Negative       True Negative

<br><br>
                        
(1.1) Dokładność = (TP + TN) / (TP + TN + FP + FN) <---- Poprawna klasyfikacja

(1.2) Precyzja = TP / (TP + FN)  <---------------------------------- Ilość poprawnie znalezionych wyników spośród tych wyświetlonych.

(1.3) Czułość = TP / (TP + FP)   <----------------------------------- Ilość znalezionych wyników spośród wszystkich pasujących.

In [3]:
dwa_tysiace_mil_podmorskiej_zeglugi = open('20_000_mil_podmorskiej_zeglugi.txt', 'r')

nemo = 0
kapitan_nemo = 0
nemo_or_ned = 0

for sentence in dwa_tysiace_mil_podmorskiej_zeglugi:
    # Poprawne wyszukanie
    if "Nemo" in sentence:
        nemo += 1
    
    # Wyszukanie, które pomija część wyników
    if "kapitan Nemo" in sentence:
        kapitan_nemo += 1

    # Wyszukanie błędne
    if "Nemo" in sentence or "Ned" in sentence:
        nemo_or_ned += 1

print(f"Precyzja niepełnego wyszukania: {round(100 * kapitan_nemo / kapitan_nemo, 2)}% \nCzułość niepełnego wyszukania: {round(100 * kapitan_nemo / nemo, 2)}%")
print()
print(f"Precyzja niepoprawnego wyszukania: {round(100 * nemo / nemo_or_ned, 2)}% \nCzułość niepoprawnego wyszukania: {round(100 * nemo / nemo, 2)}%")

Precyzja niepełnego wyszukania: 100.0% 
Czułość niepełnego wyszukania: 50.13%

Precyzja niepoprawnego wyszukania: 51.07% 
Czułość niepoprawnego wyszukania: 100.0%


### algorytm Rabina-Karpa

Wersja uproszczona bez rolowania hash'a, wyszukiwania wielu wzorców.

[Rabin-Karp algorithm Wikipedia](https://en.wikipedia.org/wiki/Rabin%E2%80%93Karp_algorithm)

In [4]:
from timeit import Timer
from functools import partial

def rabinKarp(pattern, text):
    """Simplified Rabin-Karp algorithm"""
    
    m = len(pattern)
    n = len(text)

    hpattern = hash(pattern)

    for i in range(0, n-m+1):
        htext = hash(text[i : i+m])
        
        if htext == hpattern:
            if text[i :i+m] == pattern:
                return i
    else:
        return "Not found"



text = open('20_000_mil_podmorskiej_zeglugi.txt', 'r').read()
pattern = "podług prawa natury wszystko idzie od prawej do lewej strony"

args = (pattern, text)
t = Timer(partial(rabinKarp, *args))
print("Search with Rabin-Karp:", t.timeit(number=1))

Search with Rabin-Karp: 0.11455478300194954


Zauważmy, że naiwne podejśce jest dla tego problemu znacząco szybsze. 

In [5]:
def naiveSearch(pattern, text):

    m = len(pattern)
    n = len(text)
    
    for i in range(0, n-m+1):
        if text[i :i+m] == pattern:
            return i
    else:
        return "Not found"

text = open('20_000_mil_podmorskiej_zeglugi.txt', 'r').read()
pattern = "podług prawa natury wszystko idzie od prawej do lewej strony"

args = (pattern, text)
t = Timer(partial(naiveSearch, *args))
print("Naive search:", t.timeit(number=1))

Naive search: 0.09652286599884974


Jednak generalna koncepcja algorytmu może być zastosowana do stworzenia systemu wyszukiwania informacji. 

In [6]:
import hashlib

def precomputed_hashes(hpattern, hashes):        
    return hpattern in hashes


text = open('bible.txt', 'r')

hashes = dict()
verse = ""
for paragraph in text:
    if paragraph.strip() == "":
        verse = verse.strip()

        b = bytes(verse, encoding='utf-8')
        hash_val = hashlib.sha256(b).hexdigest()
        bucket = int(hash_val, 16) % 100

        if bucket in hashes.keys():
            hashes[bucket].append(hash_val)
        else:
            hashes[bucket] = list()
            hashes[bucket].append(hash_val) 

        verse = ""
    else:
        verse += " " + paragraph.strip()

In [7]:
not_in_text_patterns = ["Oczekiwałem na kapitana Nemo, ale się wcale nie pokazał",
"Zakomunikowałem te spostrzeżenia i obawy kapitanowi Nemo",
"Poczekawszy jeszcze trochę, udałem się do salonu",
"Śród takich okoliczności płynęliśmy aż do 13 marca",
"Zakładanie dokonywało się pomyślnie",]

bible_patterns = ["1:3 And God said, Let there be light: and there was light.", 
"4:13 Therefore set I in the lower places behind the wall, and on the higher places, I even set the people after their families with their swords, their spears, and their bows.",
"37:11 But the meek shall inherit the earth; and shall delight themselves in the abundance of peace.",
"16:6 We have heard of the pride of Moab; he is very proud: even of his haughtiness, and his pride, and his wrath: but his lies shall not be so.",
"5:8 And there are three that bear witness in earth, the Spirit, and the water, and the blood: and these three agree in one."]

patterns = bible_patterns + not_in_text_patterns

def findPatterns(patterns, hashes):
    hpatterns = [hashlib.sha256(bytes(pattern, encoding='utf-8')).hexdigest() for pattern in patterns]

    for hpattern in hpatterns:
        bucket = int(hpattern, 16) % 100

        if hpattern in hashes[bucket]:
            pass
            #print("Match")
        else:
            pass
            #print("Miss")

args = (patterns, hashes)
t = Timer(partial(findPatterns, *args))
print("Multiple patterns hashed search:", t.timeit(number=1))

Multiple patterns hashed search: 0.0002994339956785552


In [8]:
text = open('bible.txt', 'r')

def naiveWholeSents(patterns, text):
    verse = ""
    for paragraph in text:
        if paragraph.strip() == "":
            verse = verse.strip()

            if verse in patterns:
                #print("Match")
                pass
            
            verse = ""
        else:
            verse += " " + paragraph.strip()
        

args = (patterns, text)
t = Timer(partial(naiveWholeSents, *args))
print("Naive search with multiple patterns:", t.timeit(number=1))

Naive search with multiple patterns: 0.06325988099706592


## 2. Wyszukiwanie pełnotekstowe

- wykorzystywanie tylko tytułów i meta-tagów
- spamdexowanie
- Która z wyszukiwarek internetowych była jako pierwsza wyszukiwarką pełnotekstową?

<img src="altavista.png" alt="AltaVista search engine" width="1000"/>

[ źródło: [https://digital.com/wp-content/uploads/4307188078_f290aecf49_o.png](https://digital.com/wp-content/uploads/4307188078_f290aecf49_o.png)]

_______________________________________________________

#### Indeksy w książkach:

<img src="book-index.jpg" alt="Indexes in books" />

[ źródło: [https://book-editing.com/why-book-indexing/](https://www.book-editing.com/why-book-indexing/) ]

#### Odwrócony indeks

- budowanie indeksu
- sortowanie
- wyszukiwanie
- dodatkowe informacje: częstość, miejsce wystąpienia
- kompresja

<img src="inverted-index.png" width=400 />

[ źródło: https://nlp.stanford.edu/IR-book/html/htmledition/a-first-take-at-building-an-inverted-index-1.html ]

Wyszukiwanie pełnotekstowe może służyć poprawie nawigacji na stronie. Gdy widzimy okienko wyszukiwania na stronie jaka technologia zasila to rozwiązanie?


- Apache Lucene i rozwiązania oparte o Lucene takie jak np. Solr- otwartoźródłowa biblioteka, najbardziej modyfikowalna opcja

- Google Programmable Search Engine, AWS CloudSearch, Elasticsearch , Algolia itp. - a więc rozwiązania od różnych dostawców takich usług

- Wyszukiwanie bezpośrednio w bazie danych lub rozwiązanie typu client-side (Czy naprawdę tego chcemy?)

----------------------------
* [How to add full text search to your website by Sam Dutton](https://medium.com/dev-channel/how-to-add-full-text-search-to-your-website-4e9c80ce2bf4)
<br><br>
* [Apache Lucene](https://lucene.apache.org/)
* [Lucene talk with tamar Syn-Hershko](https://www.youtube.com/watch?v=Nf9p-d01p78)
<br><br>
* [Google Programmable Search Engine](https://developers.google.com/custom-search)
* [AWS CloudSearch](https://aws.amazon.com/cloudsearch/)
* [Elasticsearch](https://www.elastic.co/)
<br><br>
* [Algolia vs Elasticsearch according to Algolia Part 1](https://www.algolia.com/blog/engineering/algolia-v-elasticsearch-latency/)
* [Algolia vs Elasticsearch according to Algolia Part 2](https://www.algolia.com/blog/engineering/algolia-v-elasticsearch-relevance/)
* [Inside the Algolia Engine Blog Series](https://www.algolia.com/blog/engineering/inside-the-algolia-engine-part-1-indexing-vs-search/)
* [Algolia ranking algorithm](https://www.algolia.com/blog/how-algolia-tackled-the-relevance-problem-of-search-engines/)
* [Algolia AI: vectors vs hashes](https://www.algolia.com/blog/ai/vectors-vs-hashes/)

#### przykład Algolii

<img src="algolia.svg" width=500/>

* Wyszukiwanie przyrostowe - wykorzystanie drzewa trie
* Poprawianie błędów - dystans edycyjny Damerau-Levenshtein'a
* Podkreślanie znalezionych słów kluczowych
* Lematyzacja i rozszerzanie zapytania
* Tworzenie rankingu - algorytm tie-breaking

## 3. Wyszukiwanie informacji (IR)



#### Rozszerzanie zapytania

In [9]:
from nltk.corpus import wordnet

for syn in wordnet.synsets("car"):
    print(f"meaning: {syn.definition()},\n lemmas: {syn.lemmas()}", end="\n\n")

meaning: a motor vehicle with four wheels; usually propelled by an internal combustion engine,
 lemmas: [Lemma('car.n.01.car'), Lemma('car.n.01.auto'), Lemma('car.n.01.automobile'), Lemma('car.n.01.machine'), Lemma('car.n.01.motorcar')]

meaning: a wheeled vehicle adapted to the rails of railroad,
 lemmas: [Lemma('car.n.02.car'), Lemma('car.n.02.railcar'), Lemma('car.n.02.railway_car'), Lemma('car.n.02.railroad_car')]

meaning: the compartment that is suspended from an airship and that carries personnel and the cargo and the power plant,
 lemmas: [Lemma('car.n.03.car'), Lemma('car.n.03.gondola')]

meaning: where passengers ride up and down,
 lemmas: [Lemma('car.n.04.car'), Lemma('car.n.04.elevator_car')]

meaning: a conveyance for passengers or freight on a cable railway,
 lemmas: [Lemma('cable_car.n.01.cable_car'), Lemma('cable_car.n.01.car')]



In [10]:
def query_expansion(word, meaning=0):
    query = ''
    lemmas = wordnet.synsets(word)[meaning].lemmas()
    for lemma in lemmas:

        query += " " + lemma.name()

    return query.strip()


new_query = query_expansion("car")
print(f"Query: '{new_query}'")

Query: 'car auto automobile machine motorcar'


#### Tokenizery

In [11]:
from nltk.tokenize import sent_tokenize, word_tokenize, WhitespaceTokenizer

sents = '''Good muffins cost $3.88\nin New (York).  Please (buy) me\ntwo of them.\n(Thanks).'''

tokens = []
for sent in sent_tokenize(sents):
    tokens.append(word_tokenize(sent))
    print(word_tokenize(sent))

['Good', 'muffins', 'cost', '$', '3.88', 'in', 'New', '(', 'York', ')', '.']
['Please', '(', 'buy', ')', 'me', 'two', 'of', 'them', '.']
['(', 'Thanks', ')', '.']


#### Stemming

In [12]:
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer

porter = PorterStemmer()
snowball = SnowballStemmer("english")

sent = '''If I only could, I'd be running up that hill'''

tokens = []
for token in word_tokenize(sent):
    tokens.append(token)

print([snowball.stem(token) for token in tokens])

['if', 'i', 'onli', 'could', ',', 'i', "'d", 'be', 'run', 'up', 'that', 'hill']


#### Model boolowski

* Zazwyczaj każde słowo jest interpretowane jako oddzielny token.
* Wyszukiwane wyrażenia mogą być połączone operatorami logicznymi AND, OR, a nawet NEAR.
* Konieczne stworzenie odwróconego indeksu.

In [13]:
from functools import reduce

from unittest import result
from nltk.stem.porter import *
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [14]:
class FullTextSearch:
    """Full-text search minimal example"""
    def __init__(self):
        self.inverted_index = {}
        self.stop_words = stopwords.words()
        self.stemmer = PorterStemmer()
        self.documents = {}


    def search(self, query):
        tokens = self.tokenize(query)

        results = []

        for token in tokens:
            matches = set()

            if token in self.inverted_index:
                for document in self.inverted_index[token]:
                    matches.add(document)
            
            results.append(matches)

        return reduce(lambda x, y: x & y, results)

    def add_document(self, doc, id):
        self.documents[id]  = doc
        self.index(doc, id)

    def tokenize(self, query):
        return word_tokenize(query)

    def index(self, doc, id):
        indexes = set()

        for word in self.tokenize(doc):
            if word not in self.stop_words:
                indexes.add(self.stemmer.stem(word))

        for word in indexes:
            if not word in self.inverted_index:
                self.inverted_index[word] = list()
            self.inverted_index[word].append(id)

In [15]:
from nltk.corpus import movie_reviews

file_ids = movie_reviews.fileids()
print(len(file_ids))

fts_example = FullTextSearch()

for id in file_ids[:100]:
    fts_example.add_document(movie_reviews.raw(id), id)

2000


In [16]:
problem_review = fts_example.search("problem 4/10")
cool_kid_review = fts_example.search("cool kid")

print(problem_review, end="\n")
print(cool_kid_review, end="\n")

{'neg/cv000_29416.txt'}
{'neg/cv013_10494.txt', 'neg/cv026_29229.txt', 'neg/cv000_29416.txt'}


#### Modele algebraiczne

- bag-of-words, zlicza ilość wystąpień (ang. "term frequency")
- częstość w dokumentach (ang. "document frequency")
- idf (ang. "inverse document frequency")

<img src="tf-idf.png" />

[ źródło: [https://pl.wikipedia.org/wiki/TFIDF](https://pl.wikipedia.org/wiki/TFIDF) ]

Wykorzystanie **tf-idf**:

 - Model przestrzeni wektorowej

<img src="vector-space.png" />

[ źródło: [https://www.researchgate.net/figure/A-query-and-document-representation-in-the-vector-space-model_fig1_220692395](https://www.researchgate.net/figure/A-query-and-document-representation-in-the-vector-space-model_fig1_220692395) ]

Jak mierzyć podobieństwo między dokumentami i zapytaniami?

- kosinus między wektorami

#### Modele probabilistyczne

- Probabilistyczna zasada szeregowania mówi, że dokumenty powinny być uszeregowane w kolejności prawdopodobieństwa ich użyteczności lub dopasowania. [ https://www.emerald.com/insight/content/doi/10.1108/eb026647/full/html?skipTracking=true ]

- Działają w oparciu o zasadę Bayesa (choć nie tylko: https://www.uni-mannheim.de/media/Einrichtungen/dws/Files_People/Profs/goran/5-Probabilistic-Retrieval-FSS20.pdf)

<img src="bayes.png" />

[ źródło: https://medium.com/@msalmon00/bayes-rule-and-cookies-a-primer-in-code-3ee15b6dc755 ]

- założenie Binary Independence Model, nie występują zależności między elementami. Możemy traktować każdą część wektora jako niezależną jednostkę.

- Najszerzej wykorzystywanym modelem probabilistycznym jest __BM25__.

<img src="bm25.png" />

[ źródło: [https://www.uni-mannheim.de/media/Einrichtungen/dws/Files_People/Profs/goran/5-Probabilistic-Retrieval-FSS20.pdf](https://www.uni-mannheim.de/media/Einrichtungen/dws/Files_People/Profs/goran/5-Probabilistic-Retrieval-FSS20.pdf) ]

#### Relevance feedback

<img src="relevance-feedback.jpg" width=400 />

[ źródło: (https://studfile.net/html/2706/143/html_oTTGrBg9KM._waU/htmlconvd-t8XooN216x1.jpg)[https://studfile.net/html/2706/143/html_oTTGrBg9KM._waU/htmlconvd-t8XooN216x1.jpg] ]

- algorytm Rocchia
- strona UX oceniania jakości wyników
- clickthrough data

#### Modele językowe

- Model językowy generuje rozkład prawdopodobieństwa nad słowami ze słownika. 
- Może być zrealizowany jako automat skończony, z którego możemy odczytać prawdopodobieństwa sekwencji. 
- Wygenerowany dla konkretnego dokumentu model językowy jest używany do określenia prawdopodobieństwa warunkowego na otrzymanie zapytania pod warunkiem, że zaszedł warunek wystąpienia modela językowego dla dokumentu. 


<img src="language-model.png" width=500/>
[ źródło: https://towardsdatascience.com/the-beginners-guide-to-language-models-aa47165b57f9 ]

## 4. Wyszukiwarki internetowe

Istnieją różne rozwiązania pozwalające na wyszukiwanie w sieci, wśród nich: rozwiązanie Microsoftu Bing – wbudowane w wyszukiwarkę Edge, Yahoo – posiadające kilkunastoprocentowy udział w rynku wyszukiwarek w Japonii, czy Baidu ze swoim dominującym udziałem w Chinach . Jednak niekwestionowanym liderem na globalnym rynku wyszukiwania internetowego pozostaje Google ze swoim ponad 90% udziałem.

<img src="yahoo.png" width=300/>
<img src="bing.png" width=300/>
<img src="duckduckgo.png" width=300/>
<img src="baidu.png" width=300/>

Jak działa Google według Google'a?

- [How search works](https://www.google.com/search/howsearchworks/)
- ["Szczegółowy" przewodnik po działaniu wyszukiwarki Google](https://developers.google.com/search/docs/fundamentals/how-search-works)
- [Search On 2022](https://blog.google/products/search/search-on-2022-announcements/)

Jak naprawdę działa Google? Najlepiej sprawdzić to samemu:

In [17]:
from googlesearch import search

query = 'new trends in information retrieval'
for url in search(query):
    print(url)

https://www.brainkart.com/article/Trends-in-Information-Retrieval_11614/
https://credencepressltd.com/journal/uploads/archive/202015998633983597529394.pdf
https://misc.library.ucsb.edu/untangle/lager.html
https://www.researchgate.net/publication/318530538_Trends_and_Issues_in_Modern_Information_Retrieval
https://www.igi-global.com/chapter/information-retrieval-models/198585
https://www.nowpublishers.com/INR
https://www.academia.edu/33978409/_Trends_and_issues_in_Modern_Information_Retrieval_
https://link.springer.com/chapter/10.1007/978-981-15-5554-1_14
https://www.slideshare.net/abhay.ratnaparkhi/latest-trends-inaiandinformationretrieval-autosaved
https://ieeexplore.ieee.org/document/8759000
https://pubmed.ncbi.nlm.nih.gov/5515231/


### Pająki (crawlery) internetowe

Kryteria:
    
- Pokrycie (ang. "coverage"), czyli udział witryn odwiedzonych wśród wszystkich witryn
- Świeżość (ang. "freshness"), czyli miara mówiąca nam jak dawno witryny były odwiedzane
- Pełność (ang. "completeness"), czyli jaka część zasobów z danej strony została przetworzona i zindeksowana

Dodatkowo pająk internetowy powinien cechować się uprzejmością. Uprzejmy robot powinien nie nadwyrężać zasobów strony i nie wysyłać wszystkich żądań o zasoby w jednym momencie. Pająk powinien respektować ograniczenia na niego nałożone, które są umieszczone w pliku o nazwie 'robots.txt'.

https://www.google.com/robots.txt


<img src="robots.png" alt="przykładowy plik robots.txt" />

In [18]:
from lxml import html
import requests

# https://en.wikipedia.org/wiki/List_of_HTTP_header_fields#Request_fields
headers = {
    'User-Agent': 'My tiny bot 0.1',
    'From': 'mateuszmputo@gmail.com'
}

seed = ['https://mateuszputo.github.io/contact.html']

frontier = seed
iterations = 2
for i in range(0, iterations):
    page = requests.get(frontier[0], headers=headers)
    webpage = html.fromstring(page.content)

    urls = webpage.xpath('//a/@href')
    [frontier.append(url) for url in urls]
    frontier.pop(0)

print(f"Frontier size after {iterations} iterations of crawling: {len(frontier)}")

Frontier size after 2 iterations of crawling: 105


#### Crawling stron typu RIA

Wraz z wprowadzeniem do użytku stron, które w swojej konstrukcji nie opierają się na hiperłączac między podstronami, lecz generują DOM strony dynamicznie przy użyciu skryptów JavaScript, ciężko mówić o pająkach internetowych nie wspominając o Deep-Web crawlingu.

- Jak przejść po stronie która zmienia się dynamicznie na podstawie akcji użytkownika?

Nowoczesny pająk uruchamia taką Rich Internet Application na silniku JavaScript, sprawdza czy widział daną stronę za pomocą modułu DOM-Seen i jeśli nie to przeprowadza ekstrakcję wszystkich zdarzeń do uruchomienia.

- Pytanie: Czym jest większość zdarzeń JavaScript, które próbujemy uruchomić?

<img src="deepweb-crawler.png" />

[ źródło: [https://link.springer.com/article/10.1007/s11280-018-0602-1](https://link.springer.com/article/10.1007/s11280-018-0602-1) ]

#### Zaawansowane Googlowanie:

- Wyszukiwanie frazy
- Użycie operatorów logicznych AND, OR
- Wyszukiwanie na danej stronie "site:"
- Wersja cache strony "cache:"
- wyłączenia ze znakiem minus
- Filtrowanie wyników

... żeby zostać 🥷🥷🥷 googlowania.

#### PageRank

Było na zajęciach - nie ma sensu przytaczać.

Model PageRank można interpretować jako modelujący zachowanie losowego błądzenia po stronach przez przeciętnego użytkownika.

<img src="surfer.jpg" width="400"/>

[ źródło: [https://www.surfertoday.com/surfing/how-many-surfers-are-there-in-the-world](https://www.surfertoday.com/surfing/how-many-surfers-are-there-in-the-world)]

Z ciekawostek: 

- PageRank był powodem stworzenia wyszukiwarki Google.
- Lepszy predyktor liczby cytowań od samej znanej liczby cytowań.

[ źródło: [The PageRank Citation Ranking: Bringing Order to the Web](http://ilpubs.stanford.edu:8090/422/1/1999-66.pdf)
]

#### BigTable

- NoSQL-owa baza danych. 
- Ma postać wielowymiarowej posortowanej tablicy asocjacyjnej (mapy).
- Indeksowana w oparciu o klucz rzędu i kolumny, zapisywany znacznik czasu.
- Dane przechowywane jako nieinterpretowalny ciąg bajtów.
- Pozwala na przechowywanie wielu wersji pliku w jednym rekordzie.
- Kompresja osiągająca wynik 10 do 1.

<img src="bigtable-architecture.svg" width=500/>

[ źródło: [https://cloud.google.com/bigtable/img/bigtable-architecture.svg](https://cloud.google.com/bigtable/img/bigtable-architecture.svg) ]

#### Wprowadzone zmiany

- Graf wiedzy Google
- opcja filtrowania wyników
- obsługa wielu języków
- aktualizacja zapewniająca świeżość "caffeine"
- wyszukiwanie mową oraz obrazem
- i wiele innych: https://en.wikipedia.org/wiki/Timeline_of_Google_Search

<img src="isaac-newton.png" width=700 />

#### Wykorzystanie uczenia maszynowego

Wyszukiwanie internetowe wykorzystuje duże zasoby danych. Nie jest więc dziwnym, że wiele z problemów wyszukiwania można usprawnić za pomocą rozwiązań uczenia maszynowego i uczenia głębokiego. To, co w Algolii było rozwiązywane za pomocą drzewa, czyli poprawianie literówek, Google rozwiązuje za pomocą sieci neuronowej składającej się z 680 milionów parametrów, którą udaje im się uruchomić w dwie milisekundy

- RankBrain

Pierwszym z systemów głębokiego uczenia wykorzystywanym w Googlu był RankBrain. Pomaga on w łączeniu słów i konceptów. Jak podaje Google: gdy wpiszemy zapytanie pytające o "nazwę konsumenta będącego na szczycie łańcucha pokarmowego" to algorytm RankBrain pomoże w odnalezieniu wyników zawierających wspomnienie o "drapieżniku szczytowym".

Co ciekawe gdy sprawdzimy to hasło, wyszukiwanie działa ale na razie tylko w języku angielskim.

- neural matching

Kolejnym ważnym rozwiązaniem było rozwiązanie "neural matching" ulepszające łączenie ze sobą zapytań ze stronami [42]. Działa ono prawdopodobnie wykorzystując rozszerzanie zapytania i/lub dokumentu.


- BERT

W 2019 powstał wielki model językowy BERT, który umożliwia statystyczne rozumienie tekstu . Umożliwia on prawdopodobnie ponowne szeregowanie odnalezionych wyników (ang. "re-ranking"), a wg. Googla pozwala też na niepomijanie ważnych słow, które inaczej mogłyby być uznane za słowa ze stop listy (ang. "stop words").

- MUM

Jednak największym do tej pory przedsięwzięciem jest MUM czyli "Multitask Unified Model" pozwalający na rozumienie i generowanie tekstu w 75 językach oraz rozwiązujący wiele rodzajów zadań. Model ten jest również multi modalny, może zrozumieć oprócz tekstu również obraz. Model ma być początkowo zintegrowany z Google Lens, wyszukiwarką obrazem w telefonie.

- [How AI powers great search results](https://blog.google/products/search/how-ai-powers-great-search-results/)
- [The ABCs of spelling in Google Search](https://blog.google/products/search/abcs-spelling-google-search/)
- [MUM: A new AI milestone for understanding information](https://blog.google/products/search/introducing-mum/)

## 5. Przetwarzanie języka naturalnego

Książka do bibilioteki NLP w Pythonie:

    - [NLTK Book](https://www.nltk.org/book/)

<img src="nltk.jpg" width=300 />

[ źródło: [https://static01.helion.com.pl/global/okladki/326x466/e_2gte.jpg](https://static01.helion.com.pl/global/okladki/326x466/e_2gte.jpg) ]

Wyszukiwanie informacji używa klasycznie statystycznych miar językowych do określania charakterystyk dokumentów. Dziedzina przetwarzania języka naturalnego (NLP) zajmuje się wykonywaniem manipulacji na tekście naturalnym, czyli takim, jakim posługują się ludzie na co dzień w mowie i piśmie, przy użyciu komputerów. Od prostego zliczania wystąpień i innych miar statystycznych po zrozumienie tekstu i generowanie odpowiedzi.

#### Hipoteza dystrybucyjna

Słowa o podobnym znaczeniu występują w podobnych kontekstach.

#### Dystans edycyjny

Dystans edycyjny to miara podobieństwa dwóch łańcuchów tekstowych, która jest mierzona za pomocą liczby podstawowych operacji, które należy wykonać, aby zamienić jeden z łańcuchów na drugi. Jednym z popularniejszych metryk jest odległość Levenshteina pozwalająca na operację usunięcia, dodania i zamiany znaku w łańcuchu.

## 6. Zastosowanie metod uczenia maszynowego

Zadanie wyszukiwania informacji ad-hoc polega na znalezieniu rankingu najbardziej podobnego do optymalnego opartego o prawdopodobieństwo dopasowania. Funkcja szeregowania Q × D → R oblicza wynik dopasowania dla pary dokument, zapytanie. Funkcja taka może być skonstruowana w ten sposób, że najpierw oblicza reprezentacje dla dokumentów, zapytań, par dokumentów i zapytań, a na końcu agreguje te reprezentacje. Takie systemy są systemami reprezentacyjnymi, w przeciwieństwie do systemów interakcyjnych, polegających na aktywnym przetworzeniu zapytania wraz z dokumentem, aby otrzymać wynik dopasowania.

#### Word2vec, doc2vec

- [Efficient Estimation of Word Representations in Vector Space](https://arxiv.org/pdf/1301.3781.pdf)
- [Distributed Representations of Sentences and Documents](https://arxiv.org/pdf/1405.4053.pdf)

In [19]:
from gensim.models import Word2Vec
import gensim.downloader

glove_vectors = gensim.downloader.load('glove-twitter-25')

rome_vec = glove_vectors.get_vector("paris", norm=False) - glove_vectors.get_vector("france", norm=False) + glove_vectors.get_vector("italy", norm=False)

print("Word2vec vector operations 'paris - france + italy':")
print(glove_vectors.most_similar(rome_vec))

print("Basic usage 'similar to word %pc%':")
print(glove_vectors.most_similar("pc"))

Word2vec vector operations 'paris - france + italy':
[('italy', 0.8724863529205322), ('brazil', 0.823341429233551), ('regina', 0.764968752861023), ('thailand', 0.7616036534309387), ('greet', 0.7495027184486389), ('perform', 0.743165910243988), ('demi', 0.7425128221511841), ('roma', 0.7410842180252075), ('milan', 0.7396031618118286), ('band', 0.7381407022476196)]
Basic usage 'similar to word %pc%':
[('tablet', 0.8791574835777283), ('iphone', 0.8717852830886841), ('windows', 0.867980420589447), ('ipod', 0.8656975030899048), ('internet', 0.8508771061897278), ('notebook', 0.8482425212860107), ('flash', 0.8476564884185791), ('ipad', 0.8319953083992004), ('auto', 0.8283662796020508), ('app', 0.826338529586792)]


#### Transformery

Czy chodzi o coś takiego?

<img src="transformers.jpg" />

[ źródło: [https://netflix.com](https://netflix.com)]

#### Transformery

Raczej o coś takiego:

<img src="transformer.png" />

[ źródło: [https://i.stack.imgur.com/eAKQu.png](https://i.stack.imgur.com/eAKQu.png)]

#### MS MARCO

- Jeden z najpopularniejszych zbiorów danych dla trenowania i testowania modeli wyszukiwania informacji.
- Składa się on z ponad miliona zapytań pozyskanych z logów wyszukiwarki Bing, każde z odpowiedzią wygenerowaną przez człowieka.
- Ponadto zbiór ten zawiera prawie 9 milionów paragrafów pozyskanych z ponad 3,5 miliona dokumentów. Ustępy są wybrane jako 10 odpowiedzi na pytanie ustalonych przy pomocy wyszukiwarki Bing.
- Kilka ustandaryzowanych zadań w tym ranking dokumentów oraz ranking paragrafów.
- Najlepsze wyniki osiągnięte dla zadań dostępne na stronie głównej projektu.

<img src="msmarco.png" width=500/>

[ https://microsoft.github.io/msmarco/ ]

#### Modele

- BM25 + BERT, osiągnięto poprawę 27% na miarze MRR@10.
- Doc2query, rozszerzenie dokumentu o 10 pytań (top-k sampling)
- ColBert (z faiss), dzięki architekturze bienkodera osiąga lepszą złożoność czasową.  
- HLART, efektywny trzeci stopień wyszukiwania
- PROP, dostosowany trening na modelowaniu językowym [2021 SOTA]
- Condenser i coCondenser [SOTA 2022]
    * pretrening
    * uniknięcie rozproszonej reprezentacji
    * dotrenowanie na zadaniu określania odległości między dokumentami

Prace naukowe:
- [Passage re-ranking with BERT](https://arxiv.org/pdf/1901.04085.pdf)
- [Document Expansion by Query Prediction](https://arxiv.org/pdf/1904.08375.pdf)
- [ColBERT: Efficient and Effective Passage Search via Contextualized Late Interaction over BERT](https://arxiv.org/pdf/2004.12832.pdf)
- [HLATR: Enhance Multi-stage Text Retrieval with Hybrid List Aware Transformer Reranking](https://arxiv.org/pdf/2205.10569.pdf)
- [Condenser: a Pre-training Architecture for Dense Retrieval](https://arxiv.org/pdf/2104.08253.pdf)
- [Unsupervised Corpus Aware Language Model Pre-training for Dense Passage Retrieval](https://arxiv.org/pdf/2108.05540.pdf)

#### Odpowiadanie na pytania

In [20]:
from transformers import MobileBertConfig, MobileBertModel

# Initializing a MobileBERT configuration
configuration = MobileBertConfig()

# Initializing a model (with random weights) from the configuration above
model = MobileBertModel(configuration)

# Accessing the model configuration
configuration = model.config

/home/mputo/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2022-11-30 13:41:04.407541: W tensorflow/tsl/platform/default/dso_loader.cc:66] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-30 13:41:04.407580: I tensorflow/tsl/cuda/cudart_stub.cc:28] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-30 13:41:07.055208: W tensorflow/tsl/platform/default/dso_loader.cc:66] Could not load dynamic library 'libnvinfer.so.8'; dlerror: libnvinfer.so.8: cannot open shared object file: No such file or directory
2022-11-30 13:41:07.055376: W tensorflow/tsl/platform/default/dso_loader.cc:66] Could not load dynamic library 'libnvinfer_plugin.so.8'; dlerror: libnvinfer_plugin.so.8: 

In [21]:
from transformers import MobileBertTokenizer, MobileBertForQuestionAnswering
import torch

tokenizer = MobileBertTokenizer.from_pretrained("csarron/mobilebert-uncased-squad-v2")
model = MobileBertForQuestionAnswering.from_pretrained("csarron/mobilebert-uncased-squad-v2")

text = "Barack Hussein Obama II (/bəˈrɑːk huːˈseɪn oʊˈbɑːmə/ (listen) bə-RAHK hoo-SAYN oh-BAH-mə;[1] born August 4, 1961) is an American politician who served as the 44th president of the United States from 2009 to 2017. A member of the Democratic Party, he was the first African-American president of the United States.[2] Obama previously served as a U.S. senator from Illinois from 2005 to 2008 and as an Illinois state senator from 1997 to 2004.\
Obama was born in Honolulu, Hawaii. After graduating from Columbia University in 1983, he worked as a community organizer in Chicago. In 1988, he enrolled in Harvard Law School, where he was the first black president of the Harvard Law Review. After graduating, he became a civil rights attorney and an academic, teaching constitutional law at the University of Chicago Law School from 1992 to 2004. Turning to elective politics, he represented the 13th district in the Illinois Senate from 1997 until 2004, when he ran for the U.S. Senate. Obama received national attention in 2004 with his March Senate primary win, his well-received July Democratic National Convention keynote address, and his landslide November election to the Senate. In 2008, after a close primary campaign against Hillary Clinton, he was nominated by the Democratic Party for president. Obama was elected over Republican nominee John McCain in the general election and was inaugurated alongside his running mate Joe Biden, on January 20, 2009. Nine months later, he was named the 2009 Nobel Peace Prize laureate, a decision that drew a mixture of praise and criticism."

questions = ["Where was Barack Obama born?",
            "In what year did Barack Obama become a president?",
            "What university did Obama attend?",
            "In what time-period did Obama serve as president?",
            "With whom did Barack Obama win presidential election?",
            "Who did Obama beat in presidential election?"]

for question in questions:
    inputs = tokenizer(question, text, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)

    answer_start_index = outputs.start_logits.argmax()
    answer_end_index = outputs.end_logits.argmax()

    predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
    print(tokenizer.decode(predict_answer_tokens))

honolulu, hawaii
2009
columbia university
2009 to 2017
joe biden
john mccain


### Pytania kontrolne

1. Jakie inne typy wyszukiwań są dostępne w wyszukiwarkach internetowych, oprócz wyszukiwania słów kluczowych?
2. Jakie są trzy podstawowe modele wyszukiwania informacji?
3. W oparciu o jakie technologie buduje się nowoczesne systemy wyszukiwania informacji?


# Pytania do mnie? 🤔